Baseline Modeling

Derived from B02: ML Examples

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

In [2]:
#load in the training data
df = pd.read_pickle('demo_training.pkl')

In [3]:
X = df.drop(columns = ['discharge_location'])
y = df['discharge_location'] 

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1125, stratify=y)

ValueError: Input contains NaN

In [ ]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
pd.Series(y_test).value_counts(normalize=True)

In [ ]:
sl_modles = dict(
    dummy = DummyClassifier(strategy='prior'),
    knn = KNeighborsClassifier(),
    lr = LogisticRegression(),
    svc = SVC(),
    nn = MLPClassifier(max_iter=500),
)